# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 5. Klasifikasi Decision Tree


- Nama: Rayhan Egar Sadtya Nugraha
- Kelas: PPM-D
- NIM: 225150201111014

### 1) Import Data

Praktikum kali ini menggunakan dataset [Car Evaluation Dataset](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation) dari UCI Machine Learning Repository. Dataset ini telah digunakan pada praktikum sebelumnya. Detail fitur dapat Anda pelajari pada link yang tersedia.

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [1]:
# ! wget https://archive.ics.uci.edu/dataset/19/car+evaluation

--2024-04-01 23:37:21--  https://archive.ics.uci.edu/dataset/19/car+evaluation
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138743 (135K) [text/html]
Saving to: ‘car+evaluation’

car+evaluation      100%[===================>] 135.49K   218KB/s    in 0.6s    

2024-04-01 23:37:23 (218 KB/s) - ‘car+evaluation’ saved [138743/138743]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [2]:
import pandas as pd
import numpy as np 
data = pd.read_csv('car_sample.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [3]:
data.head()

,buying,maint,lug_boot,safety,class
0,vhigh,vhigh,small,low,unacc
1,vhigh,vhigh,small,med,unacc
2,vhigh,vhigh,small,high,unacc
3,vhigh,vhigh,med,low,unacc
4,vhigh,vhigh,med,med,unacc


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**. Agar pengacakan data dilakukan secara konstan, parameter **random_state** diisi dengan nilai integer tertentu, pada praktikum ini diset 101. Kemudian, nilai indeks pada data latih dan data uji diatur ulang agar berurutan nilainya


In [4]:
from sklearn.model_selection import train_test_split
data_latih, data_uji = train_test_split(data, test_size=0.2, random_state=101)
data_latih.reset_index(drop=True)
data_uji.reset_index(drop=True)

,buying,maint,lug_boot,safety,class
0,vhigh,high,med,med,unacc
1,low,med,small,high,unacc
2,vhigh,low,big,med,unacc
3,low,vhigh,small,med,unacc
4,med,low,small,low,unacc
...,...,...,...,...,...
341,high,med,med,med,unacc
342,low,med,big,med,good
343,vhigh,vhigh,big,med,unacc
344,vhigh,low,med,low,unacc


Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 208 data, dan **data_uji** terdiri dari 52 data

In [5]:
print(data_latih.shape[0])
print(data_uji.shape[0])

1382
346


## 3) Menghitung Gini

Nilai Gini merupakan salah satu kriteria penentu variabel apa yang akan digunakan untuk membentuk cabang pada decision tree. Variabel dengan nilai Gini terbesar akan digunakan sebagai pembentukan cabang

Buatlah fungsi bernama **hitung_gini** yang berfungsi menghitung nilai Gini dari suatu nilai pada sebuah variabel

In [6]:
def hitung_gini(kolom_kelas):
    elemen, banyak = np.unique(kolom_kelas, return_counts=True)
    freq_squared = [(banyak[i]/np.sum(banyak)) ** 2 for i in range(len(elemen))]
    nilai_gini = 1 - np.sum(freq_squared)
    return nilai_gini

Buatlah fungsi bernama **gini_split** yang digunakan untuk menghitung nilai Gini keseluruhan dari sebuah variabel.

In [7]:
def gini_split(data, nama_fitur_split, nama_fitur_kelas):
    nilai, banyak = np.unique(data[nama_fitur_split], return_counts=True)
    gini_split = np.sum([(banyak[i]/np.sum(banyak)) * hitung_gini(data.where(data[nama_fitur_split]==nilai[i]).dropna()[nama_fitur_kelas]) for i in range(len(nilai))])
    return gini_split

Ujilah fungsi **gini_split** menggunakan data_latih pada variabel **buying** dan variabel kelas bernama **class**.

In [8]:
gini_split(data_latih, 'buying', 'class')

0.4498424838345615

## 4) Pembentukan pohon

Pembentukan pohon dilakukan secara rekursif. Seperti metode rekursif pada umumnya, perlu ditentukan kondisi berhenti terlebih dahulu. Kondisi berhenti pada pembentukan pohon adalah:


1.   Jika hanya ada satu kelas pada data, kembalikan kelas tersebut
2.   Jika fitur data  = 0 (tidak ada fitur yang tersisa), kembalikan kelas dari parent
3. Jika data kosong (tidak ada data), kembalikan kelas dengan frekuensi terbanyak

Selain kondisi berhenti tersebut, dilakukan pembentukan pohon secara rekursif menggunakan fungsi **buat_tree**.



In [9]:
def buat_tree(data, data_awal, daftar_fitur, nama_fitur_kelas, kelas_parent_node=None):
    
    # If there's only one class in the data (homogenous)
    if len(np.unique(data[nama_fitur_kelas])) <= 1:

        # return that class
        return np.unique(data[nama_fitur_kelas])[0]

    # If there's no data
    elif len(data)==0:

        # return majority class
        max_index = np.argmax(np.unique(data_awal[nama_fitur_kelas], return_counts=True) [1])
        return np.unique(data_awal[nama_fitur_kelas])[max_index]
    
    # If there's no more feature
    elif len(daftar_fitur)==0:

        # return kelas parent
        return kelas_parent_node

    else:

        # Recursively buat_tree()
        kelas_parent_node = np.unique(data[nama_fitur_kelas]) [np.argmax(np.unique(data[nama_fitur_kelas], return_counts=True)[1])]
        nilai_split = [ gini_split(data, fitur, nama_fitur_kelas) for fitur in daftar_fitur]
        print("Nilai GINI: ", nilai_split)
        index_fitur_terbaik = np.argmin(nilai_split)
        fitur_terbaik = daftar_fitur[index_fitur_terbaik]
        tree = {fitur_terbaik:{}}
        print("Fitur terbaik: ", fitur_terbaik, " dengan nilai GINI ", nilai_split[index_fitur_terbaik])
        print()


        daftar_fitur = [ i for i in daftar_fitur if i != fitur_terbaik] # eliminate fitur terbaik one-by-one

        # Iterate for every unique values in fitur_terbaik
        for nilai in np.unique(data[fitur_terbaik]):
            
            # Create sub data containing that one nilai only
            sub_data = data.where(data[fitur_terbaik]==nilai).dropna()
            # Create a new sub tree from sub data
            subtree = buat_tree(sub_data, data_awal, daftar_fitur, nama_fitur_kelas, kelas_parent_node)
            # Add sub tree for every nilai to main tree
            tree[fitur_terbaik][nilai] = subtree
        
        return(tree)



Buatlah tree menggunakan data latih yang tersedia

In [10]:
tree = buat_tree(data_latih, data_latih, data_latih.columns[:-1], 'class')

Nilai GINI:  [0.4498424838345615, 0.4529877610858685, 0.4597328261751287, 0.38197123728510574]
Fitur terbaik:  safety  dengan nilai GINI  0.38197123728510574

Nilai GINI:  [0.5935274302109743, 0.6083307229854543, 0.6285472851219447]
Fitur terbaik:  buying  dengan nilai GINI  0.5935274302109743

Nilai GINI:  [0.3248004700140101, 0.4983680310515173]
Fitur terbaik:  maint  dengan nilai GINI  0.3248004700140101

Nilai GINI:  [0.40304834054834054]
Fitur terbaik:  lug_boot  dengan nilai GINI  0.40304834054834054

Nilai GINI:  [0.47586206896551725]
Fitur terbaik:  lug_boot  dengan nilai GINI  0.47586206896551725

Nilai GINI:  [0.3297777777777778]
Fitur terbaik:  lug_boot  dengan nilai GINI  0.3297777777777778

Nilai GINI:  [0.6109835674644468, 0.6525153609831029]
Fitur terbaik:  maint  dengan nilai GINI  0.6109835674644468

Nilai GINI:  [0.5197132616487455]
Fitur terbaik:  lug_boot  dengan nilai GINI  0.5197132616487455

Nilai GINI:  [0.5176136363636363]
Fitur terbaik:  lug_boot  dengan nilai

Tampilkan tree yang terbentuk. Gunakan library **pprint** untuk menampilkan dictionary secara teratur.

In [11]:
from pprint import pprint
pprint(tree)

{'safety': {'high': {'buying': {'high': {'maint': {'high': {'lug_boot': {'big': 'acc',
                                                                         'med': 'acc',
                                                                         'small': 'acc'}},
                                                   'low': {'lug_boot': {'big': 'acc',
                                                                        'med': 'acc',
                                                                        'small': 'acc'}},
                                                   'med': {'lug_boot': {'big': 'acc',
                                                                        'med': 'acc',
                                                                        'small': 'acc'}},
                                                   'vhigh': 'unacc'}},
                                'low': {'maint': {'high': {'lug_boot': {'big': 'unacc',
                                                    

## 5) Proses prediksi

Proses prediksi kelas pada data uji dilakukan dengan melakukan *tree traversal* sampai menemui leaf.

In [12]:
def prediksi(data_uji, tree):
    for key in list(data_uji.keys()):
        if key in list(tree.keys()):
            try:
                hasil = tree[key][data_uji[key]]
            except:
                return 1
            
            hasil = tree[key][data_uji[key]]
            if isinstance(hasil, dict):
                return prediksi(data_uji, hasil)
            else:
                return hasil

## 6) Proses Pengujian
Lakukan pengujian menggunakan data uji. Kelas pada data uji perlu dihapus dan data uji perlu diubah menjadi dictionary

In [13]:
data_uji_dict = data_uji.iloc[:, :-1].to_dict(orient="records")
print(data_uji_dict)

[{'buying': 'vhigh', 'maint': 'high', 'lug_boot': 'med', 'safety': 'med'}, {'buying': 'low', 'maint': 'med', 'lug_boot': 'small', 'safety': 'high'}, {'buying': 'vhigh', 'maint': 'low', 'lug_boot': 'big', 'safety': 'med'}, {'buying': 'low', 'maint': 'vhigh', 'lug_boot': 'small', 'safety': 'med'}, {'buying': 'med', 'maint': 'low', 'lug_boot': 'small', 'safety': 'low'}, {'buying': 'med', 'maint': 'high', 'lug_boot': 'med', 'safety': 'med'}, {'buying': 'med', 'maint': 'med', 'lug_boot': 'med', 'safety': 'med'}, {'buying': 'high', 'maint': 'low', 'lug_boot': 'small', 'safety': 'med'}, {'buying': 'med', 'maint': 'low', 'lug_boot': 'big', 'safety': 'high'}, {'buying': 'med', 'maint': 'med', 'lug_boot': 'med', 'safety': 'low'}, {'buying': 'vhigh', 'maint': 'vhigh', 'lug_boot': 'big', 'safety': 'low'}, {'buying': 'low', 'maint': 'vhigh', 'lug_boot': 'big', 'safety': 'med'}, {'buying': 'low', 'maint': 'low', 'lug_boot': 'big', 'safety': 'med'}, {'buying': 'high', 'maint': 'vhigh', 'lug_boot': 'b

Lakukan pengujian terhadap keseluruhan data uji menggunakan looping.

In [14]:
hasil_prediksi_total = []

for i in range(len(data_uji_dict)):
    hasil_prediksi = prediksi(data_uji_dict[i], tree)
    hasil_prediksi_total.append(hasil_prediksi)

Bandingkan hasil prediksi dengan label sebenarnya. Hitunglah banyaknya data uji yang memiliki kelas prediksi sama dengan kelas sebenarnya

In [15]:
print("Total prediksi benar: ", sum(hasil_prediksi_total==data_uji['class']))
print("Tree model accuracy: ", sum(hasil_prediksi_total==data_uji['class'])/data_uji.shape[0])

Total prediksi benar:  251
Tree model accuracy:  0.7254335260115607


## TUGAS
Pada tugas kali ini Anda diminta memodifikasi metode pembentukan tree yang telah Anda agar metode tersebut menggunakan information gain sebagai dasar percabangan. Lengkapilah kerangka source code di bawah ini

Lengkapi fungsi hitung_entropy

In [16]:
def hitung_entropy(kolom_kelas):
  elemen, banyak = np.unique(kolom_kelas, return_counts=True)
  freq_log = [(banyak[i]/np.sum(banyak)) * np.log2(banyak[i]/np.sum(banyak)) for i in range(len(elemen))]
  entropy = -1 * np.sum(freq_log)
  return entropy

Lengkapi fungsi information_gain

In [17]:
def information_gain(data, nama_fitur_split, nama_fitur_kelas):
  nilai, banyak = np.unique(data[nama_fitur_split], return_counts=True)
  node_entropy = hitung_entropy(nama_fitur_kelas)
  information_gain = node_entropy - np.sum([(banyak[i]/np.sum(banyak)) * hitung_entropy(data.where(data[nama_fitur_split]==nilai[i]).dropna()[nama_fitur_kelas]) for i in range(len(nilai))])
  return information_gain

In [18]:
# test information_gain()
information_gain(data_latih, 'buying', 'class')

-1.1235412105809561

Lengkapi fungsi **buat_tree_ig**. Isinya sama persis dengan fungsi **buat_tree**, hanya saja penghitungan **gini_split** diganti dengan **information_gain**. Selain itu, percabangan dilakukan dengan menggunakan nilai **information_gain** **terbesar**

In [19]:
def buat_tree_ig(data,data_awal, daftar_fitur, nama_fitur_kelas,kelas_parent_node=None):
  # If there's only one class in the data (homogenous)
  if len(np.unique(data[nama_fitur_kelas])) <= 1:

    # return that class
    return np.unique(data[nama_fitur_kelas])[0]

  # If there's no data
  elif len(data)==0:

    # return parent's majority class
    max_index = np.argmax(np.unique(data_awal[nama_fitur_kelas], return_counts=True) [1])
    return np.unique(data_awal[nama_fitur_kelas])[max_index]
    
  # If there's no more feature
  elif len(daftar_fitur)==0:

    # return kelas parent
    return kelas_parent_node

  else:

    # Recursively buat_tree_ig()
    kelas_parent_node = np.unique(data[nama_fitur_kelas]) [np.argmax(np.unique(data[nama_fitur_kelas], return_counts=True)[1])]
    
    # Splitting criteria dihitung berdasarkan nilai information_gain
    nilai_split = [ information_gain(data, fitur, nama_fitur_kelas) for fitur in daftar_fitur]
    print("Nilai Information Gain: ", nilai_split)
    # Fitur terbaik = fitur dengan nilai information_gain terbesar
    index_fitur_terbaik = np.argmax(nilai_split)
    fitur_terbaik = daftar_fitur[index_fitur_terbaik]
    print("Fitur terbaik: ", fitur_terbaik, " dengan nilai Information Gain ", nilai_split[index_fitur_terbaik])
    print()
    tree = {fitur_terbaik:{}}

    daftar_fitur = [ i for i in daftar_fitur if i != fitur_terbaik] # eliminate fitur terbaik one-by-one

    # Iterate for every unique values in fitur_terbaik
    for nilai in np.unique(data[fitur_terbaik]):
            
      # Create sub data containing that one nilai only
      sub_data = data.where(data[fitur_terbaik]==nilai).dropna()
      # Create a new sub tree from sub data
      subtree = buat_tree_ig(sub_data, data_awal, daftar_fitur, nama_fitur_kelas, kelas_parent_node)
      # Add sub tree for every nilai to main tree
      tree[fitur_terbaik][nilai] = subtree
      
  return(tree)

Lakukan pembentukan tree menggunakan fungsi **buat_tree_ig**

In [20]:
tree_ig = buat_tree_ig(data_latih,data_latih,data_latih.columns[:-1],'class')

Nilai Information Gain:  [-1.1235412105809561, -1.145214375382843, -1.192011002521087, -0.947072085632132]
Fitur terbaik:  safety  dengan nilai Information Gain  -0.947072085632132

Nilai Information Gain:  [-1.4297291146097422, -1.497897328795924, -1.5349453108119397]
Fitur terbaik:  buying  dengan nilai Information Gain  -1.4297291146097422

Nilai Information Gain:  [-0.6753855741523577, -0.99764351739384]
Fitur terbaik:  maint  dengan nilai Information Gain  -0.6753855741523577

Nilai Information Gain:  [-0.8453736940981362]
Fitur terbaik:  lug_boot  dengan nilai Information Gain  -0.8453736940981362

Nilai Information Gain:  [-0.964626498311624]
Fitur terbaik:  lug_boot  dengan nilai Information Gain  -0.964626498311624

Nilai Information Gain:  [-0.725558619496705]
Fitur terbaik:  lug_boot  dengan nilai Information Gain  -0.725558619496705

Nilai Information Gain:  [-1.415418223447856, -1.6119351096639847]
Fitur terbaik:  maint  dengan nilai Information Gain  -1.415418223447856

N

Nilai Information Gain:  [-1.0588649295919605]
Fitur terbaik:  lug_boot  dengan nilai Information Gain  -1.0588649295919605

Nilai Information Gain:  [-0.9458810600548044]
Fitur terbaik:  lug_boot  dengan nilai Information Gain  -0.9458810600548044

Nilai Information Gain:  [-1.2378137809202165, -1.62418521887466]
Fitur terbaik:  maint  dengan nilai Information Gain  -1.2378137809202165

Nilai Information Gain:  [-0.9273303762721083]
Fitur terbaik:  lug_boot  dengan nilai Information Gain  -0.9273303762721083

Nilai Information Gain:  [-1.0616342365529594]
Fitur terbaik:  lug_boot  dengan nilai Information Gain  -1.0616342365529594

Nilai Information Gain:  [-1.1813281794354422]
Fitur terbaik:  lug_boot  dengan nilai Information Gain  -1.1813281794354422

Nilai Information Gain:  [-0.9611664662522772]
Fitur terbaik:  lug_boot  dengan nilai Information Gain  -0.9611664662522772

Nilai Information Gain:  [-0.4704139333988096, -0.9178701517489929]
Fitur terbaik:  maint  dengan nilai Infor

Tampilkan tree yang terbentuk

In [21]:
pprint(tree_ig)

{'safety': {'high': {'buying': {'high': {'maint': {'high': {'lug_boot': {'big': 'acc',
                                                                         'med': 'acc',
                                                                         'small': 'acc'}},
                                                   'low': {'lug_boot': {'big': 'acc',
                                                                        'med': 'acc',
                                                                        'small': 'acc'}},
                                                   'med': {'lug_boot': {'big': 'acc',
                                                                        'med': 'acc',
                                                                        'small': 'acc'}},
                                                   'vhigh': 'unacc'}},
                                'low': {'maint': {'high': {'lug_boot': {'big': 'unacc',
                                                    

Lakukan pengujian menggunakan tree yang terbentuk

In [22]:
hasil_prediksi_total_ig = []
for i in range(len(data_uji_dict)):
  hasil_prediksi = prediksi(data_uji_dict[i],tree_ig)
  hasil_prediksi_total_ig.append(hasil_prediksi)
print("Total prediksi benar: ",sum(hasil_prediksi_total_ig==data_uji['class']))
print("Tree model accuracy: ", sum(hasil_prediksi_total==data_uji['class'])/data_uji.shape[0])


Total prediksi benar:  251
Tree model accuracy:  0.7254335260115607


# PERTANYAAN

Jawablah pertanyaan di bawah ini



1.   Amati tree yang dihasilkan dengan kriteria percabangan GINI dan Information Gain. Apa perbedaan tree yang dihasilkan dari kedua metode tersebut?
2.   Apakah penggunaan Information Gain dapat meningkatkan akurasi prediksi?



Tulis jawaban Anda di cell ini


1.   Pada kasus di atas dengan random_state=101, tidak ada perbedaan model antara tree yang dihasilkan dengan kriteria percabangan GINI dengan tree yang dihasilkan dengan kriteria percabangan Information Gain. Hal ini menandakan jika order _feature importance_ dari skor GINI dan Information Gain untuk semua feature dataset konsisten. Kedua model tersebut konsisten menempatkan fitur 'safety' sebagai percabangan pertama setelah _root_ node.
2.   Pada kasus di atas, tidak ada pengaruh yang diberikan pada akurasi model Decision Tree yang dibentuk dalam penggunaan Information Gain jika dibandingkan dengan penggunaan GINI sebagai splitting criteria. Akan tetapi, secara teoretikal, penggunaan GINI sebagai splitting criteria dinilai lebih baik dalam dataset yang berbentuk kategorikal (seperti pada dataset di atas) dikarenakan sifatnya yang berbasis pada impuritas/homogenitas kelas sub-data yang dihasilkan. Sedangkan penggunaan Information Gain sebagai splitting criteria dinilai lebih baik dalam dataset dengan nilai kontinu, memberikan model Decision Tree dengan depth antara _child_ yang lebih seimbang.

